In [30]:
#-*- coding: utf-8 -*-
import csv
import pandas as pd
import random
from datetime import datetime

rdf = pd.read_csv('노선.csv')
tdf = pd.read_csv('교통수단코드.csv')

day = '20181104'
f = 'C:/Data/20181229_인천시_교통카드데이터/1-1.거래내역/DI_TBTRDD010_' + day + '.csv'
df = pd.read_csv(f, index_col=None, header=0, encoding='utf-8', low_memory=False)
of = 'IC_ETick_' + day + '.txt'
df.sample(n=10).head()

,카드번호,운행출발일시,트랜잭션ID,교통수단코드,환승횟수,노선ID,교통사업자ID,차량ID,사용자구분코드,승차일시,승차정류장ID,하차일시,하차정류장ID,이용객수,통행거리,통행시간
6488278,N6TaIDKTS48Io7q6yifA9V04cMzAZUHxQ4hDGvKc+U8=,20181104154112,53,115,0,11110953,111007400,111741732,1,20181104161159,11642,20181104163350,0070064,1,5060,1311
908317,l2os9qF7sPMHCfImyLKo+37yQS1OBNYizD0LMNLlE6k=,~,1,201,0,~,211000000,~,1,20181104173648,212,20181104181104,0429,1,12600,2056
7697740,JpGhIO+nUToJX66gwGb54w==,20181104174737,12,105,1,11110976,111520042,111755259,1,20181104181105,9035078,20181104181430,9003341,1,1166,205
7275466,T3vMh86568BdxXBkMh9iB1ujNSy/Tk9rv2u0Edv2CmI=,20181104125809,26,115,0,41110088,111006600,111742323,1,20181104131529,72280,20181104132500,0008609,1,3225,571
8026978,ojG+nTPlT1PcD2TRoShuFi79mhPVMBt2mLAbsAebYp8=,20181104053342,56,105,1,11110607,111500870,111758825,1,20181104184248,9136735,20181104185620,9012664,1,2569,812


In [31]:
with open(of, mode='w', newline='') as visum_file:
    visum_writer = csv.writer(visum_file, delimiter=';', quoting=csv.QUOTE_NONE)
    
    visum_writer.writerow(['$VISION'])
    visum_writer.writerow(['* VisumInst'])
    visum_writer.writerow(['* 22.05.12, 18.27.29'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['* Tabelle: Versionsblock'])
    visum_writer.writerow(['$VERSION', 'VERSNR', 'FILETYPE', 'LANGUAGE', 'UNIT'])
    visum_writer.writerow(['4.00','Att','Eng','KM'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    visum_writer.writerow(['* Table: PuT path legs'])
    visum_writer.writerow(['$SingleRowSurveyData:No', 'NumPass', 'SurveyLineName', 'InputStopNo', 'InputStopDepTime', 'InputStopDepDay', 'BoardStopNo', 'AlightStopNo', 'DestStopNo', 'OrigStopNo', 'PreTSysCode'])
# 번호	승객수	탑승노선	탑승정류장	탑승시간	탑승일자	하차정류장	환승정류장	최종하차정류장	최초출발정류장	
    visum_writer.writerow(['*'])
    visum_writer.writerow(['*'])
    
    seq_no = 1
    no_dict = {}
    
    for index, row in df.iterrows():
        No = no_dict.setdefault(row['카드번호'], seq_no)
        if No == seq_no:
            seq_no = seq_no + 1
        
        NumPass = row['이용객수']
#         노선ID => 노선번호, 지하철 => 교통수단코드 기준 교통수단명
        SurveyLineName = row['노선ID']
        if SurveyLineName == '~':
            trans_name = tdf.loc[tdf['교통수단코드'] == row['교통수단코드'], '교통수단명']
            if len(trans_name) > 0:
#                 print('길이:', len(trans_name), '수단명: ', trans_name)
                SurveyLineName = trans_name.values[0]
            else:
                SurveyLineName = row['교통수단코드']
        else:
            route_name = rdf.loc[rdf['노선ID'] == row['노선ID'], '노선명']
            if len(route_name) > 0:
                SurveyLineName = route_name.values[0]
        InputStopNo = row['승차정류장ID']
        dt = str(row['승차일시'])
        InputStopDepTime = datetime.strptime(dt[8:], '%H%M%S').strftime('%H:%M:%S')
        InputStopDepDay = 1
#         InputStopDepDay = datetime.strptime(dt[:8], '%Y%m%d').strftime('%Y-%m-%d')
        BoardStopNo = row['승차정류장ID']
        AlightStopNo = row['하차정류장ID']
        DestStopNo = row['하차정류장ID']
        OrigStopNo = row['승차정류장ID']
        PreTSysCode = ''
        NewLine = ''
        
        visum_writer.writerow([No, NumPass, SurveyLineName, InputStopNo, 
                              InputStopDepTime, InputStopDepDay, BoardStopNo, AlightStopNo, 
                              DestStopNo, OrigStopNo, PreTSysCode, NewLine])